# Dispatch problems with `pypsa`

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa highspy
```
:::

PyPSA stands for **Python for Power System Analysis**.

PyPSA is an open source Python package for simulating and optimising modern energy systems that include features such as

- conventional generators with unit commitment (ramp-up, ramp-down, start-up, shut-down),
- time-varying wind and solar generation,
- energy storage with efficiency losses and inflow/spillage for hydroelectricity
- coupling to other energy sectors (electricity, transport, heat, industry),
- conversion between energy carriers (e.g. electricity to hydrogen),
- transmission networks (AC, DC, other fuels)

PyPSA can be used for a variety of problem types (e.g. electricity market modelling, long-term investment planning, transmission network expansion planning), and is designed to scale well with large networks and long time series.

Compared to building power system by hand in `linopy`, PyPSA does the following things for you:

- manage data inputs
- build optimisation problem
- communicate with the solver
- retrieve and process optimisation results
- manage data outputs

### Dependencies

- `pandas` for storing data about network components and time series
- `numpy` and `scipy` for linear algebra and sparse matrix calculations
- `matplotlib` and `cartopy` for plotting on a map
- `networkx` for network calculations
- `linopy` for handling optimisation problems

:::{note}
Documentation for this package is available at https://pypsa.readthedocs.io.
:::

## Basic Structure

| Component | Description |
| --- | --- |
| [Network](https://pypsa.readthedocs.io/en/latest/components.html#network) | Container for all components. |
| [Bus](https://pypsa.readthedocs.io/en/latest/components.html#bus) | Node where components attach. |
| [Carrier](https://pypsa.readthedocs.io/en/latest/components.html#carrier) | Energy carrier or technology (e.g. electricity, hydrogen, gas, coal, oil, biomass, on-/offshore wind, solar). Can track properties such as specific carbon dioxide emissions or nice names and colors for plots. |
| [Load](https://pypsa.readthedocs.io/en/latest/components.html#load) | Energy consumer (e.g. electricity demand). |
| [Generator](https://pypsa.readthedocs.io/en/latest/components.html#generator) | Generator (e.g. power plant, wind turbine, PV panel). |
| [Line](https://pypsa.readthedocs.io/en/latest/components.html#line) | Power distribution and transmission lines (overhead and cables). |
| [Link](https://pypsa.readthedocs.io/en/latest/components.html#link) | Links connect two buses with controllable energy flow, direction-control and losses. They can be used to model: <ul><li>HVDC links</li><li>HVAC lines (neglecting KVL, only net transfer capacities (NTCs))</li><li>conversion between carriers (e.g. electricity to hydrogen in electrolysis)</li></ul> |
| [StorageUnit](https://pypsa.readthedocs.io/en/latest/components.html#storage-unit) | Storage with fixed nominal energy-to-power ratio. |
| [GlobalConstraint](https://pypsa.readthedocs.io/en/latest/components.html#global-constraints) | Constraints affecting many components at once, such as emission limits. |
| [Store](https://pypsa.readthedocs.io/en/latest/components.html#store) | Storage with separately extendable energy capacity. |
|  | **not used in this course** | 
| [LineType](https://pypsa.readthedocs.io/en/latest/components.html#line-types) | Standard line types. |
| [Transformer](https://pypsa.readthedocs.io/en/latest/components.html#transformer) | 2-winding transformer. |
| [TransformerType](https://pypsa.readthedocs.io/en/latest/components.html#transformer-types) | Standard types of 2-winding transformer. |
| [ShuntImpedance](https://pypsa.readthedocs.io/en/latest/components.html#shunt-impedance) | Shunt. |


:::{note}
Links in the table lead to documentation for each component.
:::

<img src="https://pypsa.readthedocs.io/en/latest/_images/buses.png" width="500px" />


:::{warning}
Per unit values of voltage and impedance are used internally for network calculations. It is assumed internally that the base power is **1 MW**.
:::

## From structured data to optimisation

The design principle of PyPSA is that basically each component is associated with a set of variables and constraints that will be added to the optimisation model based on the input data stored for the components. 

For an *hourly* electricity market simulation, PyPSA will solve an optimisation problem that looks like this

\begin{equation}
\min_{g_{i,s,t}; f_{\ell,t}; g_{i,r,t,\text{charge}}; g_{i,r,t,\text{discharge}}; e_{i,r,t}} \sum_s o_{s} g_{i,s,t}
\end{equation}
such that
\begin{align}
0 & \leq g_{i,s,t} \leq \hat{g}_{i,s,t} G_{i,s}  & \text{generation limits : generator} \\
-F_\ell &\leq f_{\ell,t} \leq F_\ell & \text{transmission limits : line}  \\
d_{i,t} &= \sum_s g_{i,s,t} + \sum_r g_{i,r,t,\text{discharge}} - \sum_r g_{i,r,t,\text{charge}} - \sum_\ell K_{i\ell} f_{\ell,t} & \text{KCL : bus} \\
 0 &=\sum_\ell C_{\ell c} x_\ell f_{\ell,t} & \text{KVL : cycles} \\
0 & \leq g_{i,r,t,\text{discharge}} \leq G_{i,r,\text{discharge}}& \text{discharge limits : storage unit} \\
    0 & \leq g_{i,r,t,\text{charge}} \leq G_{i,r,\text{charge}} & \text{charge limits : storage unit} \\
    0 & \leq e_{i,r,t} \leq E_{i,r} & \text{energy limits : storage unit} \\ 
    e_{i,r,t} &= \eta^0_{i,r,t} e_{i,r,t-1} + \eta^1_{i,r,t}g_{i,r,t,\text{charge}} -  \frac{1}{\eta^2_{i,r,t}} g_{i,r,t,\text{discharge}} & \text{consistency : storage unit} \\
    e_{i,r,0} & = e_{i,r,|T|-1}  & \text{cyclicity : storage unit}
\end{align}

**Decision variables:**

- $g_{i,s,t}$ is the generator dispatch at bus $i$, technology $s$, time step $t$,
- $f_{\ell,t}$ is the power flow in line $\ell$,
- $g_{i,r,t,\text{dis-/charge}}$ denotes the charge and discharge of storage unit $r$ at bus $i$ and time step $t$,
- $e_{i,r,t}$ is the state of charge of storage $r$ at bus $i$ and time step $t$.

**Parameters:**

- $o_{i,s}$ is the marginal generation cost of technology $s$ at bus $i$,
- $x_\ell$ is the reactance of transmission line $\ell$,
- $K_{i\ell}$ is the incidence matrix,
- $C_{\ell c}$ is the cycle matrix,
- $G_{i,s}$ is the nominal capacity of the generator of technology $s$ at bus $i$,
- $F_{\ell}$ is the rating of the transmission line $\ell$,
- $E_{i,r}$ is the energy capacity of storage $r$ at bus $i$,
- $\eta^{0/1/2}_{i,r,t}$ denote the standing (0), charging (1), and discharging (2) efficiencies.

:::{note}
For a full reference to the optimisation problem description, see https://pypsa.readthedocs.io/en/latest/optimal_power_flow.html
:::

## Simple electricity market example

Let's get acquainted with PyPSA by building simple electricity market models.

We have the following data:

- fuel costs in € / MWh$_{th}$ 

In [70]:
fuel_cost = dict(
    coal=8,
    gas=100,
    oil=48,
    hydro=1,
    wind=0,
)

- efficiencies of thermal power plants in MWh$_{el}$ / MWh$_{th}$

In [71]:
efficiency = dict(
    coal=0.33,
    gas=0.58,
    oil=0.35,
    hydro=1,
    wind=1,
)

- specific emissions in t$_{CO_2}$ / MWh$_{th}$

In [72]:
# t/MWh thermal
emissions = dict(
    coal=0.34,
    gas=0.2,
    oil=0.26,
    hydro=0,
    wind=0,
)

- power plant capacities in MW

In [73]:
power_plants = {
    "A": {"coal": 35000, "wind": 3000, "gas": 8000, "oil": 2000},
    "B": {"hydro": 1200},
}

- perfectly inelastic electrical load in MW

In [74]:
loads = {
    "A": 42000,
    "B": 650,
}

## Building a basic network

By convention, PyPSA is imported without an alias:

In [75]:
import pypsa

First, we create a new network object which serves as the overall container for all components.

In [76]:
n = pypsa.Network()

The second component we need are buses. **Buses** are the fundamental nodes of the network, to which all other components like loads, generators and transmission lines attach. They enforce energy conservation for all elements feeding in and out of it (i.e. Kirchhoff’s Current Law).

Components can be added to the network `n` using the `n.add()` function. It takes the component name as a first argument, the name of the component as a second argument and possibly further parameters as keyword arguments. Let's use this function, to add buses for each country to our network:

In [77]:
n.add("Bus", "A", v_nom=400, carrier="AC")
n.add("Bus", "B", v_nom=400, carrier="AC")

Index(['B'], dtype='object')

For each class of components, the data describing the components is stored in a `pandas.DataFrame`. For example, all static data for buses is stored in `n.buses`

In [78]:
n.buses

,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
A,400.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,
B,400.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,


You see there are many more attributes than we specified while adding the buses; many of them are filled with default parameters which were added. You can look up the field description, defaults and status (required input, optional input, output) for buses here https://pypsa.readthedocs.io/en/latest/components.html#bus, and analogous for all other components. 

The method `n.add()` also allows you to add multiple components at once. For instance, multiple **carriers** for the fuels with information on specific carbon dioxide emissions, a nice name, and colors for plotting. For this, the function takes the component name as the first argument and then a list of component names and then optional arguments for the parameters. Here, scalar values, lists, dictionary or `pandas.Series` are allowed. The latter two needs keys or indices with the component names.

In [79]:
n.add(
    "Carrier",
    ["coal", "gas", "oil", "hydro", "wind"],
    co2_emissions=emissions,
    nice_name=["Coal", "Gas", "Oil", "Hydro", "Onshore Wind"],
    color=["grey", "indianred", "black", "aquamarine", "dodgerblue"],
)

Index(['coal', 'gas', 'oil', 'hydro', 'wind'], dtype='object')

The `n.add()` function is very general. It lets you add any component to the network object `n`. For instance, in the next step we add **generators** for all the different power plants.

In Region B:

In [80]:
n.add(
    "Generator",
    "B hydro",
    bus="B",
    carrier="hydro",
    p_nom=1200,  # MW
)

Index(['B hydro'], dtype='object')

In Region A (in a loop):

In [81]:
for tech, p_nom in power_plants["A"].items():
    n.add(
        "Generator",
        f"A {tech}",
        bus="A",
        carrier=tech,
        efficiency=efficiency.get(tech, 1),
        p_nom=p_nom,
        marginal_cost=fuel_cost.get(tech, 0) / efficiency.get(tech, 1),
    )

As a result, the `n.generators` DataFrame looks like this:

In [82]:
n.generators

,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
B hydro,B,PQ,,1200.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
A coal,A,PQ,,35000.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
A wind,A,PQ,,3000.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
A gas,A,PQ,,8000.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
A oil,A,PQ,,2000.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


Next, we're going to add the electricity demand.

A positive value for `p_set` means consumption of power from the bus.

In [83]:
n.add(
    "Load",
    "A electricity demand",
    bus="A",
    p_set=loads["A"],
)

Index(['A electricity demand'], dtype='object')

In [84]:
n.add(
    "Load",
    "B electricity demand",
    bus="B",
    p_set=loads["B"],
)

Index(['B electricity demand'], dtype='object')

In [85]:
n.loads

,bus,carrier,type,p_set,q_set,sign,active
Load,,,,,,,
A electricity demand,A,,,42000.0,0.0,-1.0,True
B electricity demand,B,,,650.0,0.0,-1.0,True


Finally, we add the connection between Region B and Region A with a 500 MW line:

In [86]:
n.add(
    "Line",
    "A-B",
    bus0="A",
    bus1="B",
    s_nom=500,
    x=1,
    r=1,
)

Index(['A-B'], dtype='object')

In [87]:
n.lines

,bus0,bus1,type,x,r,g,b,s_nom,s_nom_mod,s_nom_extendable,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
A-B,A,B,,1.0,1.0,0.0,0.0,500.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Optimisation

With all input data transferred into PyPSA's data structure, we can now build and run the resulting optimisation problem. In PyPSA, building, solving and retrieving results from the optimisation model is contained in a single function call `n.optimize()`. This function optimizes dispatch and investment decisions for least cost.

The `n.optimize()` function can take a variety of arguments. The most relevant for the moment is the choice of the solver (e.g. "highs" and "gurobi"). They need to be installed on your computer, to use them here!

In [88]:
n.optimize(solver_name="highs")

Index(['A', 'B'], dtype='object', name='Bus')
Index(['A-B'], dtype='object', name='Line')
Index(['A', 'B'], dtype='object', name='Bus')
Index(['A-B'], dtype='object', name='Line')
/home/fneum/miniconda3/envs/nordnet/lib/python3.12/site-packages/linopy/common.py:147: UserWarning:

coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /tmp/highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 6 primals, 19 duals
Objective: 1.38e+06
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper were not 

Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+01, 2e+02]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 4e+04]
Presolving model
1 rows, 2 cols, 2 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-19); columns 0(-6); elements 0(-24) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  1.3813912524e+06
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-9cxaq8k8.sol


('ok', 'optimal')

Let's have a look at the results.

Since the power flow and dispatch are generally time-varying quantities, these are stored in a different location than e.g. `n.generators`. They are stored in `n.generators_t`. Thus, to find out the dispatch of the generators, run

In [89]:
n.generators_t.p

Generator,B hydro,A coal,A wind,A gas,A oil
snapshot,,,,,
now,1150.0,35000.0,3000.0,1500.0,2000.0


or if you prefer it in relation to the generators nominal capacity

In [90]:
n.generators_t.p / n.generators.p_nom

Generator,B hydro,A coal,A wind,A gas,A oil
snapshot,,,,,
now,0.958333,1.0,1.0,0.1875,1.0


You see that the time index has the value 'now'. This is the default index when no time series data has been specified and the network only covers a single state (e.g. a particular hour). 

Similarly you will find the power flow in transmission lines at

In [91]:
n.lines_t.p0

Line,A-B
snapshot,
now,-500.0


In [92]:
n.lines_t.p1

Line,A-B
snapshot,
now,500.0


The `p0` will tell you the flow from `bus0` to `bus1`. `p1` will tell you the flow from `bus1` to `bus0`.

What about the _market clearing prices_ in the electricity market?

In [93]:
n.buses_t.marginal_price

Bus,A,B
snapshot,,
now,172.413793,-0.0


## Modifying networks

Modifying data of components in an existing PyPSA network is as easy as modifying the entries of a `pandas.DataFrame`. For instance, if we want to reduce the cross-border transmission capacity between Region A and Region B, we'd run:

In [94]:
n.lines.loc["A-B", "s_nom"] = 400

In [95]:
n.lines

,bus0,bus1,type,x,r,g,b,s_nom,s_nom_mod,s_nom_extendable,...,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,v_nom
Line,,,,,,,,,,,,,,,,,,,,,
A-B,A,B,,1.0,1.0,0.0,0.0,400.0,0.0,False,...,inf,0,0.000006,0.000006,0.0,0.0,0.000006,0.000006,500.0,400.0


In [96]:
n.optimize(solver_name="highs")

Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
Index(['A-B'], dtype='object', name='Line')
Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
Index(['A-B'], dtype='object', name='Line')
/home/fneum/miniconda3/envs/nordnet/lib/python3.12/site-packages/linopy/common.py:147: UserWarning:

coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /tmp/highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 6 primals, 19 duals
Objective: 1.40e+06
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constr

Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+01, 2e+02]
  Bound  [0e+00, 0e+00]
  RHS    [4e+02, 4e+04]
Presolving model
1 rows, 2 cols, 2 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-19); columns 0(-6); elements 0(-24) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  1.3986326317e+06
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-eagrmxnj.sol


('ok', 'optimal')

You can see that the production of the hydro power plant was reduced and that of the gas power plant increased owing to the reduced transmission capacity.

In [97]:
n.generators_t.p

Generator,B hydro,A coal,A wind,A gas,A oil
snapshot,,,,,
now,1050.0,35000.0,3000.0,1600.0,2000.0


## Global constraints for emission limits

In the example above, we happen to have some spare gas capacity with lower carbon intensity than the coal and oil generators. We could use this to lower the emissions of the system, but it will be more expensive. We can implement the limit of carbon dioxide emissions as a constraint.

This is achieved in PyPSA through **Global Constraints** which add constraints that apply to many components at once.

But first, we need to calculate the current level of emissions to set a sensible limit.

We can compute the emissions per generator (in tonnes of CO$_2$) in the following way.

$$\frac{g_{i,s,t} \cdot \rho_{i,s}}{\eta_{i,s}}$$

where $ \rho$ is the specific emissions (tonnes/MWh thermal) and $\eta$ is the conversion efficiency (MWh electric / MWh thermal) of the generator with dispatch $g$ (MWh electric):

In [98]:
e = (
    n.generators_t.p
    / n.generators.efficiency
    * n.generators.carrier.map(n.carriers.co2_emissions)
)
e

Generator,B hydro,A coal,A wind,A gas,A oil
snapshot,,,,,
now,0.0,36060.606061,0.0,551.724138,1485.714286


Summed up, we get total emissions in tonnes:

In [99]:
e.sum().sum()

38098.044484251375

So, let's say we want to reduce emissions by 10%:

In [100]:
n.add(
    "GlobalConstraint",
    "emission_limit",
    carrier_attribute="co2_emissions",
    sense="<=",
    constant=e.sum().sum() * 0.9,
)

Index(['emission_limit'], dtype='object')

In [101]:
n.optimize(solver_name="highs")

Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
Index(['A-B'], dtype='object', name='Line')
Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
Index(['A-B'], dtype='object', name='Line')
/home/fneum/miniconda3/envs/nordnet/lib/python3.12/site-packages/linopy/common.py:147: UserWarning:

coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.25s
INFO:linopy.solvers:Log file at /tmp/highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 6 primals, 20 duals
Objective: 2.12e+06
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constr

Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-01, 1e+00]
  Cost   [2e+01, 2e+02]
  Bound  [0e+00, 0e+00]
  RHS    [4e+02, 4e+04]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
1 rows, 2 cols, 2 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-20); columns 0(-6); elements 0(-27) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  2.1206212849e+06
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-mpql59ib.sol


('ok', 'optimal')

In [102]:
n.generators_t.p

Generator,B hydro,A coal,A wind,A gas,A oil
snapshot,,,,,
now,1050.0,30603.423345,3000.0,7996.576655,-0.0


In [103]:
n.generators_t.p / n.generators.p_nom

Generator,B hydro,A coal,A wind,A gas,A oil
snapshot,,,,,
now,0.875,0.874384,1.0,0.999572,-0.0


In [104]:
n.global_constraints.mu

GlobalConstraint
emission_limit   -216.158537
Name: mu, dtype: float64

Can we lower emissions even further? Say by another 5% points?

In [105]:
n.global_constraints.loc["emission_limit", "constant"] = 0.85 * e.sum().sum()

In [106]:
n.optimize(solver_name="highs")

Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
Index(['A-B'], dtype='object', name='Line')
Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
Index(['A-B'], dtype='object', name='Line')
/home/fneum/miniconda3/envs/nordnet/lib/python3.12/site-packages/linopy/common.py:147: UserWarning:

coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /tmp/highs.log
Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: infeasible



Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-01, 1e+00]
  Cost   [2e+01, 2e+02]
  Bound  [0e+00, 0e+00]
  RHS    [4e+02, 4e+04]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
1 rows, 2 cols, 2 nonzeros  0s
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-1tecf4f4.sol


('warning', 'infeasible')

No! Without any additional capacities, we have exhausted our options to reduce CO2 in that hour. The optimiser tells us that the problem is *infeasible*.

## Exercises

For the exercises, let's first remove the emission limit again:

In [107]:
n.remove("GlobalConstraint", "emission_limit")

**Task 1:** Model an outage of the transmission line by setting it's capacity to zero. How does the model compensate for the lack of transmission? 

In [108]:
n.lines["s_nom"] = 0.
n.optimize()
n.generators_t.p

Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
Index(['A-B'], dtype='object', name='Line')
Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
Index(['A-B'], dtype='object', name='Line')
/home/fneum/miniconda3/envs/nordnet/lib/python3.12/site-packages/linopy/common.py:147: UserWarning:

coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /tmp/highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 6 primals, 19 duals
Objective: 1.47e+06
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constr

Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+01, 2e+02]
  Bound  [0e+00, 0e+00]
  RHS    [6e+02, 4e+04]
Presolving model
1 rows, 2 cols, 2 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-19); columns 0(-6); elements 0(-24) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  1.4675981490e+06
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-qv8u4vd6.sol


Generator,B hydro,A coal,A wind,A gas,A oil
snapshot,,,,,
now,650.0,35000.0,3000.0,2000.0,2000.0


**Task 2:** Double the wind capacity. How is the electricity price affected? What generator is price-setting?

In [109]:
n.generators.loc["A wind", "p_nom"] *= 2
n.optimize()
n.buses_t.marginal_price

Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
Index(['A-B'], dtype='object', name='Line')
Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
Index(['A-B'], dtype='object', name='Line')
/home/fneum/miniconda3/envs/nordnet/lib/python3.12/site-packages/linopy/common.py:147: UserWarning:

coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /tmp/highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 6 primals, 19 duals
Objective: 9.86e+05
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constr

Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+01, 2e+02]
  Bound  [0e+00, 0e+00]
  RHS    [6e+02, 4e+04]
Presolving model
1 rows, 3 cols, 3 nonzeros  0s
1 rows, 3 cols, 3 nonzeros  0s
Presolve : Reductions: rows 1(-18); columns 3(-3); elements 3(-21)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1(36000) 0s
          1     9.8562770563e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  9.8562770563e+05
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-bc1j8x6h.sol


Bus,A,B
snapshot,,
now,137.142857,-0.0


**Task 3:** Region A brings a new 1000 MW nuclear power plant into operation with an estimated marginal electricity generation cost of 20 €/MWh. Will that affect the electricity price?

In [110]:
n.add(
    "Generator",
    "A nuclear",
    bus="A",
    carrier="nuclear",
    p_nom=1000,
    marginal_cost=20,
)
n.optimize()
n.buses_t.marginal_price

Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')


Index(['A-B'], dtype='object', name='Line')
Index(['A nuclear'], dtype='object', name='Generator')
Index(['A', 'B'], dtype='object', name='Bus')
Index(['0'], dtype='object', name='SubNetwork')
Index(['A-B'], dtype='object', name='Line')
Index(['A nuclear'], dtype='object', name='Generator')
/home/fneum/miniconda3/envs/nordnet/lib/python3.12/site-packages/linopy/common.py:147: UserWarning:

coords for dimension(s) ['Generator'] is not aligned with the pandas object. Previously, the indexes of the pandas were ignored and overwritten in these cases. Now, the pandas object's coordinates are taken considered for alignment.

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.solvers:Log file at /tmp/highs.log
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 7 primals, 22 duals
Objective: 8.68e+05
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-pric

Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+01, 2e+02]
  Bound  [0e+00, 0e+00]
  RHS    [6e+02, 4e+04]
Presolving model
1 rows, 3 cols, 3 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-22); columns 0(-7); elements 0(-28) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  8.6848484848e+05
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-leggr_kf.sol


Bus,A,B
snapshot,,
now,24.242424,-0.0
